In [1]:
from gmip.utils import  analytical_tpr,fn_dong
from ipywidgets import widgets, Layout
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import trapz 
import math

In [19]:
def plot_x(mu=1):
    w = np.linspace(0, 1, 100)
    plt.ylabel("TPR")
    plt.xlabel("FPR")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
  
    plt.grid(color='0.6')
    plt.gcf().set_size_inches(3.0,3.0)
    plt.plot(w,analytical_tpr(w,mu))
    plt.title("Trade-off curve")

def calc_privacy_lvl(C, tau, T, n, N, d):
    n_eff = n + (n*n*tau*tau)/(C*C)
    mu_step = (d+(2*n-1))/(n_eff*math.sqrt(2*d + 4*((n*n)/n_eff)))
    c = (n*math.sqrt(T))/N
    mu_tot =  c*fn_dong(mu_step)
    return mu_tot

def floater(C):
    try:
        C = math.inf if C == 'inf' else float(C)
    except ValueError:
        C = 1.0
    return C

 Notes: 
 - The cropping threshold C only has an effect, when tau > 0.0.
 - Also have finite privacy without noise, when tau = 0.0.

In [21]:
def wrapper(C, tau, T, n, N, d):
    C = float(C)
    result = calc_privacy_lvl(C, tau, T, n, N, d)
    fpr_values = np.linspace(0, 1, 100)
    tpr_values = [analytical_tpr(fpr, result) for fpr in fpr_values]
    au_roc = trapz(tpr_values, fpr_values)
    tpr_at_001 = tpr_values[np.argmin(np.abs(fpr_values - 0.01))]
    tpr_at_005 = tpr_values[np.argmin(np.abs(fpr_values - 0.05))]
    acc = 0.5 * (tpr_values + (1 - fpr_values))
    
    print(f"Privacy Level: {result}")
    print(f"AU-ROC: {au_roc}")
    print(f"TPR@0.01: {tpr_at_001}")
    print(f"TPR@0.05: {tpr_at_005}")
    print('Accuracy:', np.max(acc))
    
    fig, ax = plt.subplots()
    ax.plot(fpr_values, tpr_values, label='TPR vs. FPR')
    ax.fill_between(fpr_values, tpr_values, step='mid', alpha=0.4)
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('ROC Curve')
    ax.legend()
    plt.show()

style = {'description_width': 'initial'}
layout = {'width': '400px'}
C_widget = widgets.Text(value='inf', description='Cropping threshold (C):', style=style, layout=layout)
tau_widget = widgets.FloatText(value=0, description='Tau (Noise):', style=style, layout=layout)
T_widget = widgets.IntText(value=10, description='T (SGD Steps):', style=style, layout=layout)
n_widget = widgets.IntText(value=10, description='n (Dataset Size):', style=style, layout=layout)
N_widget = widgets.IntText(value=10, description='N (Number in Dataset):', style=style, layout=layout)
d_widget = widgets.IntText(value=2, description='d (Model Parameters):', style=style, layout=layout)

ui = widgets.VBox([
    widgets.HTML('<h3>How Private is Your Model?</h3>'),
    C_widget, tau_widget, T_widget, n_widget, N_widget, d_widget
])

out = widgets.interactive_output(wrapper, {'C': C_widget, 'tau': tau_widget, 'T': T_widget, 'n': n_widget, 'N': N_widget, 'd': d_widget})

display(ui, out)

Output()